# STEPS units helper functions

Helper functions to convert between the conventions used in STEPS
and those used in appendix A of https://link.springer.com/978-3-319-63113-4

## Convert between STEPS concentration (mol/litre) and population count

In [1]:
# convert STEPS concentration (mol/litre) in specified volume (m^3) to a count of molecules
def get_count(concentration, volume):
    A = 6.02214076e23  # Avogadro number: molecules in 1 mol
    litres = 1e3 * volume
    return A * concentration * litres


# convert count of molecules to a STEPS concentration (mol/litre) in specified volume (m^3)
def get_concentration(count, volume):
    A = 6.02214076e23  # Avogadro number: molecules in 1 mol
    mols = count / A
    litres = 1e3 * volume
    return mols / litres

## Convert between STEPS `kcst` ($M^{1-n}s^{-1}$) and population stochastic reaction rate $c$ in a given volume ($s^{-1}$)

The propensity for a reaction $j$ in https://link.springer.com/978-3-319-63113-4 is given by

$a_j(x) = c_j h_j(x)$,

$h_j(x) = \prod_i x_i!/[v_{ji}^{-}!(x_i - v_{ji}^{-})!]$, where
- $c_j$ has units of $s^{-1}$
- $x_i$ is the *number* of molecules of type $i$ currently in the system
- $v_{ji}^{-}$ is the number of molecules of type $i$ on the left hand side of reaction $j$.

In STEPS, the propensity is given by

$a_j(x) = k_j h_j(x)$,

$h_j(x) = \prod_i x_i!/[(x_i - v_{ji}^{-})!]$, where

- $k_j$ is the parameter `kcst` for the reaction $j$, which has units $M^{1-v_j}s^{-1}$
- $x_i$ is the *concentration* ($M$ = mol/litre) of molecules of type $i$ currently in the system
- $v_j \equiv \sum_i v_{ji}$ is the order of the reaction

So comparing the two, to convert from $c$ to `kcst` we need a multiplicative factor to compensate for both the different units and the missing factorial factor in the definition of $h$:

In [2]:
def get_rate_conversion_factor(lhs_molecule_list, volume):
    import numpy as np

    # Avogadro number: molecules in 1 mol
    A = 6.02214076e23
    litres = 1e3 * volume
    # get order of reaction
    order = len(lhs_molecule_list)
    # get factorial factor
    counts = dict()
    factorial_factor = 1
    for lhs in lhs_molecule_list:
        counts[lhs] = counts.get(lhs, 0) + 1
        factorial_factor *= counts[lhs]
    return factorial_factor * np.power(A * litres, 1 - order)


# convert stochastic reaction rate in specified volume to STEPS kcst
def get_kcst(stoch_rate_c, lhs_molecule_list, volume):
    factor = get_rate_conversion_factor(lhs_molecule_list, volume)
    return stoch_rate_c / factor


# convert STEPS kcst to stochastic reaction rate in specified volume
def get_stoch_rate(stoch_rate_kcst, order, volume):
    factor = get_rate_conversion_factor(lhs_molecule_list, volume)
    return stoch_rate_kcst * factor

## Tests

We create a model to check the conversion functions are working

In [3]:
import steps.geom as swm
import steps.model as smodel
import steps.rng as srng
import steps.solver as ssolver

mdl = smodel.Model()
vsys = smodel.Volsys('vsys', mdl)

molS = smodel.Spec('S', mdl)
molT = smodel.Spec('T', mdl)
molU = smodel.Spec('U', mdl)


# some reactions from p13, example 2.3 of https://link.springer.com/978-3-319-63113-4
kreac_synthesis = smodel.Reac('kreac_synthesis', vsys, lhs=[], rhs=[molS])
kreac_unimolecular = smodel.Reac('kreac_unimolecular', vsys, lhs=[molS], rhs=[molT])
kreac_bimolecular = smodel.Reac('kreac_bimolecular', vsys, lhs=[molS, molT], rhs=[molU])
kreac_dimerization = smodel.Reac(
    'kreac_dimerization', vsys, lhs=[molS, molS], rhs=[molT]
)
kreac_polymerization = smodel.Reac(
    'kreac_polymerization', vsys, lhs=[molT, molT, molT], rhs=[molU]
)
kreac_termomolecular = smodel.Reac(
    'kreac_termomolecular', vsys, lhs=[molS, molT, molT], rhs=[molU]
)

wmgeom = swm.Geom()
comp = swm.Comp('comp', wmgeom)
comp.addVolsys('vsys')
comp.setVol(1e-20)

rng = srng.create('mt19937', 256)

solver = ssolver.Wmdirect(mdl, wmgeom, rng)

### Check of count <--> concentration conversions

In [4]:
solver.reset()
volume = 1.123e-21
solver.setCompVol('comp', volume)

count -> concentration:

In [5]:
count = 100
concentration = get_concentration(count, volume)
solver.setCompConc('comp', 'S', concentration)
print("desired count:\t%f" % count)
print("actual count:\t%f" % solver.getCompCount('comp', 'S'))

desired count:	100.000000
actual count:	100.000000


concentration -> count:

In [6]:
concentration = 0.2341
count = get_count(concentration, volume)
solver.setCompCount('comp', 'S', count)
print("desired concentration:\t%f" % concentration)
print("actual concentration:\t%f" % solver.getCompConc('comp', 'S'))

desired concentration:	0.234100
actual concentration:	0.234100


### Check of stochastic rate -> `kcst` conversion

In [7]:
volume = 3.44e-22
solver.setCompVol('comp', volume)

Synthesis: $\emptyset \rightarrow S$

In [8]:
def a_synthesis(count, c):
    return c


c = 0.1264
k = get_kcst(c, [], volume)
solver.reset()
solver.setCompReacK('comp', 'kreac_synthesis', k)
print("count\tdesired a\tactual a")
for count in [0, 1, 2, 3, 100]:
    solver.setCompCount('comp', 'S', count)
    print(
        "%d\t%f\t%f"
        % (count, a_synthesis(count, c), solver.getCompReacA('comp', 'kreac_synthesis'))
    )

count	desired a	actual a
0	0.126400	0.126400
1	0.126400	0.126400
2	0.126400	0.126400
3	0.126400	0.126400
100	0.126400	0.126400


Unimolecular: $S \rightarrow T$

In [9]:
def a_unimolecular(count, c):
    return count * c


c = 0.56854
k = get_kcst(c, ['S'], volume)
solver.reset()
solver.setCompReacK('comp', 'kreac_unimolecular', k)
print("count\tdesired a\tactual a")
for count in [0, 1, 2, 3, 100]:
    solver.setCompCount('comp', 'S', count)
    print(
        "%d\t%f\t%f"
        % (
            count,
            a_unimolecular(count, c),
            solver.getCompReacA('comp', 'kreac_unimolecular'),
        )
    )

count	desired a	actual a
0	0.000000	0.000000
1	0.568540	0.568540
2	1.137080	1.137080
3	1.705620	1.705620
100	56.854000	56.854000


Bimolecular: $S + T \rightarrow U$

In [10]:
def a_bimolecular(countS, countT, c):
    return countS * countT * c


c = 3.56854
k = get_kcst(c, ['S', 'T'], volume)
solver.reset()
solver.setCompReacK('comp', 'kreac_bimolecular', k)
print("count S\tcount T\tdesired a\tactual a")
for countS in [0, 1, 2, 3, 100]:
    for countT in [0, 1, 2, 3, 100]:
        solver.setCompCount('comp', 'S', countS)
        solver.setCompCount('comp', 'T', countT)
        print(
            "%d\t%d\t%f\t%f"
            % (
                countS,
                countT,
                a_bimolecular(countS, countT, c),
                solver.getCompReacA('comp', 'kreac_bimolecular'),
            )
        )

count S	count T	desired a	actual a
0	0	0.000000	0.000000
0	1	0.000000	0.000000
0	2	0.000000	0.000000
0	3	0.000000	0.000000
0	100	0.000000	0.000000
1	0	0.000000	0.000000
1	1	3.568540	3.568539
1	2	7.137080	7.137079
1	3	10.705620	10.705618
1	100	356.854000	356.853939
2	0	0.000000	0.000000
2	1	7.137080	7.137079
2	2	14.274160	14.274158
2	3	21.411240	21.411236
2	100	713.708000	713.707878
3	0	0.000000	0.000000
3	1	10.705620	10.705618
3	2	21.411240	21.411236
3	3	32.116860	32.116855
3	100	1070.562000	1070.561817
100	0	0.000000	0.000000
100	1	356.854000	356.853939
100	2	713.708000	713.707878
100	3	1070.562000	1070.561817
100	100	35685.400000	35685.393897


Dimerization: $2S \rightarrow T$

In [11]:
def a_dimerization(count, c):
    return count * (count - 1) * c / 2


c = 0.3312585685678
k = get_kcst(c, ['S', 'S'], volume)
solver.reset()
solver.setCompReacK('comp', 'kreac_dimerization', k)
print("count\tdesired a\tactual a")
for count in [0, 1, 2, 3, 100]:
    solver.setCompCount('comp', 'S', count)
    print(
        "%d\t%f\t%f"
        % (
            count,
            a_dimerization(count, c),
            solver.getCompReacA('comp', 'kreac_dimerization'),
        )
    )

count	desired a	actual a
0	0.000000	0.000000
1	0.000000	0.000000
2	0.331259	0.331259
3	0.993776	0.993776
100	1639.729914	1639.729634


Polymerization: $3T \rightarrow U$

In [12]:
def a_polymerization(count, c):
    return count * (count - 1) * (count - 2) * c / 6


c = 0.467835624523
k = get_kcst(c, ['T', 'T', 'T'], volume)
solver.reset()
solver.setCompReacK('comp', 'kreac_polymerization', k)
print("count\tdesired a\tactual a")
for count in [0, 1, 2, 3, 100]:
    solver.setCompCount('comp', 'T', count)
    print(
        "%d\t%f\t%f"
        % (
            count,
            a_polymerization(count, c),
            solver.getCompReacA('comp', 'kreac_polymerization'),
        )
    )

count	desired a	actual a
0	0.000000	0.000000
1	0.000000	0.000000
2	0.000000	0.000000
3	0.467836	0.467835
100	75649.020485	75648.994608


Termolecular: $S + 2T \rightarrow U$

In [13]:
def a_termolecular(countS, countT, c):
    return countS * countT * (countT - 1) * c / 2


c = 2.4675362431
k = get_kcst(c, ['S', 'T', 'T'], volume)
solver.reset()
solver.setCompReacK('comp', 'kreac_termomolecular', k)
print("count S\tcount T\tdesired a\tactual a")
for countS in [0, 1, 2, 3, 100]:
    for countT in [0, 1, 2, 3, 100]:
        solver.setCompCount('comp', 'S', countS)
        solver.setCompCount('comp', 'T', countT)
        print(
            "%d\t%d\t%f\t%f"
            % (
                countS,
                countT,
                a_termolecular(countS, countT, c),
                solver.getCompReacA('comp', 'kreac_termomolecular'),
            )
        )

count S	count T	desired a	actual a
0	0	0.000000	0.000000
0	1	0.000000	0.000000
0	2	0.000000	0.000000
0	3	0.000000	0.000000
0	100	0.000000	0.000000
1	0	0.000000	0.000000
1	1	0.000000	0.000000
1	2	2.467536	2.467535
1	3	7.402609	7.402606
1	100	12214.304403	12214.300225
2	0	0.000000	0.000000
2	1	0.000000	0.000000
2	2	4.935072	4.935071
2	3	14.805217	14.805212
2	100	24428.608807	24428.600450
3	0	0.000000	0.000000
3	1	0.000000	0.000000
3	2	7.402609	7.402606
3	3	22.207826	22.207819
3	100	36642.913210	36642.900676
100	0	0.000000	0.000000
100	1	0.000000	0.000000
100	2	246.753624	246.753540
100	3	740.260873	740.260620
100	100	1221430.440335	1221430.022519
